In [1]:
from bs4 import BeautifulSoup
import requests

In [ ]:
1.coding=>2L52.1a.1,2RSSE.1a.1
2.non_coding => 2L52.2 biotype=ncRNA,B0212.t1 biotype=tRNA,AC8.8 biotype=pre_miRNA
例外:
VT23B5.1
F56F12.8
Y17G7A.2
3.pseudogenic_transcripts=>B0238.13a,2RSSE.3,AC3.9   問題解決:下載的fasta strand會不對所以序列不一樣
4.Transposon-mRNA=>
H05L03.5 type=Transposon-non-coding_transcript 這個可以(是因為這個剛好屬於nc_rna),
B0250.13 type=Transposon-pseudogenic_transcript 這種型態的似乎不行,
B0302.6  type=Transposon-mRNA,這種型態的似乎不行,
F59C6.1, type=Transposon-mRNA 這種型態的似乎不行,

In [55]:
import requests
import json
import ssl
import pandas as pd
from IPython.display import display, JSON

# 伪造请求头
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0 Safari/537.36'
}

# 忽略 SSL 验证 (如果需要)
# ssl._create_default_https_context = ssl._create_unverified_context

# 读取 CSV 文件
combined_df = pd.read_csv('combined_data.csv')

# 输入 transcript_id
transcript_id = input("Enter transcript ID: ")

# 查找对应的类型
transcript_info = combined_df[combined_df['Transcript_Name'] == transcript_id]

# 检查是否找到了对应的 transcript
if not transcript_info.empty:
    transcript_type = transcript_info.iloc[0]['Type']
    if transcript_type in {'pseudogenic_transcripts', 'Transposon-pseudogenic_transcript'}:
    # 處理 pseudogenic_transcripts 或 Transposon-pseudogenic_transcript 的邏輯

        # 构建伪基因（pseudogene）的请求 URL
        url = f'https://wormbase.org/rest/widget/pseudogene/{transcript_id}/sequences'
    elif transcript_type == 'Transposon-mRNA':
        # 构建 Transposon-mRNA 的请求 URL
        url = f'https://wormbase.org/rest/widget/cds/{transcript_id}/sequences'
    else:
        # 构建编码转录本（coding transcript）的请求 URL
        url = f'https://wormbase.org/rest/widget/transcript/{transcript_id}/sequences'
    
    # 发送请求并获取响应
    response = requests.get(url, headers=headers)

    # 检查响应状态码
    if response.status_code == 200:
        # 解析 JSON 数据
        data = response.json()
        # 显示 JSON 数据
        display(JSON(data))
    else:
        print(f"Failed to fetch data: {response.status_code}")
else:
    print(f"Transcript ID '{transcript_id}' not found in the CSV.")


<IPython.core.display.JSON object>

In [56]:
formatted_json = json.dumps(data, indent=3)
print(formatted_json)

{
   "name": "B0302.6",
   "uri": "rest/widget/cds/B0302.6/sequences",
   "class": "cds",
   "fields": {
      "cds_sequence": {
         "description": "the spliced sequence of the transcripts without UTR",
         "data": {
            "negative_strand": {
               "features": [
                  {
                     "type": "exon",
                     "start": 1,
                     "stop": 250
                  },
                  {
                     "type": "exon",
                     "start": 251,
                     "stop": 1218
                  }
               ],
               "sequence": "ATGCTCAAAAAGTGTGCGACACCGCTCGCGTTACCCCTCTCTATTATTTACTCCCAGTCATTGAAAGAATCAACCATTCCCTCCATTTGGAAGCATGCCACTGTCATTCCTGTACCCAAAAAAGGCAACCCCTCAAATCCATCTAATTATCGTCCAATTAGCTTAACTGATACTATTGCTAGAGTAATGGAGAAAATTATTTCACACCGAAGTCTGAATACGGTCACCTCATATCCAAACATCAACACGGTCAAAATCAATTCCACAGTTTCCGCTAATGCCCTCCCCATTCCTTCTGGGGTGCCCCAAGGAACAGTTACAGGTCCCCTTCTCTTTATATTGTTTTTGAACGACCTACTCCTATCCATACCCCCT

In [57]:
if transcript_type in {'pseudogenic_transcripts', 'Transposon-pseudogenic_transcript'}:
    # 對於 pseudogene 的邏輯
    table= data['fields']['predicted_exon_structure']['data']
    if data['fields']['sequence_context']['data']['strand']=='+':
        sequence=data['fields']['sequence_context']['data']['positive_strand']['sequence']
    else:
        sequence=data['fields']['sequence_context']['data']['negative_strand']['sequence']
elif transcript_type == 'Transposon-mRNA':
    if data['fields']['cds_sequence']['data']['strand'] == '+':
        table = data['fields']['cds_sequence']['data']['positive_strand']['features']
        sequence = data['fields']['cds_sequence']['data']['positive_strand']['sequence']
    else:
        table = data['fields']['cds_sequence']['data']['negative_strand']['features']
        sequence = data['fields']['cds_sequence']['data']['negative_strand']['sequence']
else:
    # 處理 coding 或 non-coding 的邏輯
    if data['fields']['spliced_sequence_context']['data']['strand'] == '+':
        table = data['fields']['spliced_sequence_context']['data']['positive_strand']['features']
        sequence = data['fields']['spliced_sequence_context']['data']['positive_strand']['sequence']
    else:
        table = data['fields']['spliced_sequence_context']['data']['negative_strand']['features']
        sequence = data['fields']['spliced_sequence_context']['data']['negative_strand']['sequence']
    if not table:
        table = data['fields']['predicted_exon_structure']['data']

In [58]:
table,sequence

([{'type': 'exon', 'start': 1, 'stop': 250},
  {'type': 'exon', 'start': 251, 'stop': 1218}],
 'ATGCTCAAAAAGTGTGCGACACCGCTCGCGTTACCCCTCTCTATTATTTACTCCCAGTCATTGAAAGAATCAACCATTCCCTCCATTTGGAAGCATGCCACTGTCATTCCTGTACCCAAAAAAGGCAACCCCTCAAATCCATCTAATTATCGTCCAATTAGCTTAACTGATACTATTGCTAGAGTAATGGAGAAAATTATTTCACACCGAAGTCTGAATACGGTCACCTCATATCCAAACATCAACACGGTCAAAATCAATTCCACAGTTTCCGCTAATGCCCTCCCCATTCCTTCTGGGGTGCCCCAAGGAACAGTTACAGGTCCCCTTCTCTTTATATTGTTTTTGAACGACCTACTCCTATCCATACCCCCTTCTATTCAATTTTCTTGTTTTGCTGACGATGTGAAAATATTCAGTCACGATCCAATTGAACTTCAAAGCACAATTGATCTCATTGTAGATTGGTCTTTGAAAAATCAGTTGCCTCTTGCTCCCACCAAAACGGCCCTACTCCACATAGGAACTCAAAACCCTAGGCATAATTTCAATATTGACTCCCTACCCATCGTCCCCAGTTCCTCTGTTAGAGACCTCGGGCTTATCACCGACGATAAATTGTCATTCAAATCGCATATCAATAGAACCTCTTCCCTGGCACTTCTTAGGGCAAAACAGATCCTAAAATCCTTCCCCTCTGGCTCCAATAAATTCTATGTGAACCTGTTTAAAACGTATGTCTCACCAATTCTTGACTATGGGTCCCACATTTACTCCCCTCCCCCTCATTCAAAACTGTCAGGTGCATTAGAATCCGCTCTCAAATACTACTCAAGAATAATACTACAACGTAACAATATACAATACAAAGTATCGGAAACATCACATAGAAACACCTCATTAG

In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

# 創建 Service 對象，傳遞 chromedriver 的路徑
service = Service(executable_path="chromedriver-win64/chromedriver.exe")

# 使用 Service 對象來初始化 Chrome WebDriver
driver = webdriver.Chrome(service=service)
driver.get('https://www.facebook.com/?locale=zh_TW')

time.sleep(2)

# 輸入帳號密碼並登入
email = driver.find_element(By.NAME, 'email')
password = driver.find_element(By.NAME, 'pass')
email.send_keys('ruby.wenlii@hotmail.com')
password.send_keys('268473951RANGERS')

time.sleep(2)

button = driver.find_element(By.NAME, 'login')
button.click()

time.sleep(5)  # 等待頁面加載完成

# 查找搜尋列並輸入關鍵字
search = driver.find_element(By.NAME, 'x1a2a7pz x1qjc9v5 xnwf7zb x40j3uw x1s7lred x15gyhx8 x9f619 x78zum5 x1fns5xo x1n2onr6 xh8yej3 x1ba4aug xmjcpbm')
search.click()
time.sleep(2)
search.send_keys("Computational Systems Biology Lab")
search.send_keys(Keys.RETURN)  # 模擬按下 Enter 鍵

time.sleep(5)  # 等待搜尋結果加載

# 在這裡添加任何你想要對搜尋結果執行的操作

# 關閉瀏覽器
driver.quit()


ElementClickInterceptedException: Message: element click intercepted: Element <input dir="ltr" aria-autocomplete="list" aria-expanded="false" role="combobox" placeholder="搜尋 Facebook" autocomplete="off" spellcheck="false" aria-invalid="false" aria-label="搜尋 Facebook" class="x1i10hfl xggy1nq x1s07b3s x1kdt53j x1yc453h xhb22t3 xb5gni xcj1dhv x2s2ed0 xq33zhf xjyslct xjbqb8w xnwf7zb x40j3uw x1s7lred x15gyhx8 x972fbf xcfux6l x1qhh985 xm0m39n x9f619 xzsf02u xdl72j9 x1iyjqo2 xs83m0k xjb2p0i x6prxxf xeuugli x1a2a7pz x1n2onr6 x15h3p50 xm7lytj x1sxyh0 xdvlbce xurb0ha x1vqgdyp x1xtgk1k x17hph69 xo6swyp x1ad04t7 x1glnyev x1ix68h3 x19gujb8" type="search" value=""> is not clickable at point (108, 28). Other element would receive the click: <div class="x1uvtmcs x4k7w5x x1h91t0o x1beo9mf xaigb6o x12ejxvf x3igimt xarpa2k xedcshv x1lytzrv x1t2pt76 x7ja8zs x1n2onr6 x1qrby5j x1jfb8zj" tabindex="-1">...</div>
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF629A8EEA2+31554]
